In [1]:
import scala.io.Source

val filePath = "data/the_verdict.txt"
val rawText = Source.fromFile(filePath).mkString

println(s"Read ${rawText.length} characters from $filePath")

Read 20479 characters from data/the_verdict.txt


import scala.io.Source
filePath: String = "data/the_verdict.txt"
rawText: String = """I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)

"The height of his glory"--that was what the women called it. I can hear Mrs. Gideon Thwing--his last Chicago sitter--deploring his unaccountable abdication. "Of course it's going to send the value of my picture 'way up; but I don't think of that, Mr. Rickham--the loss to Arrt is all I think of." The word, on Mrs. Thwing's lips, multiplied its _rs_ as though they were reflected in an endless vista of mirrors. And it was not only the Mrs. Thwings who mourned. Had not the exquisite Hermia Croft, at the last Grafton Gallery show, stopped me before Gisburn's "Moon-dancers" to say, wi

In [2]:
def tokenize(text: String): Vector[String] = {
  val splitBy = """[,.:;?_!"()\']|--|\s"""
  text.split(s"(?<=$splitBy)|(?=$splitBy)").filter(!_.isBlank).toVector
}

println(tokenize("Hello, world. Is this-- a test?"))

Vector(Hello, ,, world, ., Is, this, --, a, test, ?)


defined function tokenize

In [3]:
val tokenizedText = tokenize(rawText)

println(s"Extracted ${tokenizedText.length} tokens")

Extracted 4690 tokens


tokenizedText: Vector[String] = Vector(
  "I",
  "HAD",
  "always",
  "thought",
  "Jack",
  "Gisburn",
  "rather",
  "a",
  "cheap",
  "genius",
  "--",
  "though",
  "a",
  "good",
  "fellow",
  "enough",
  "--",
  "so",
  "it",
  "was",
  "no",
  "great",
  "surprise",
  "to",
  "me",
  "to",
  "hear",
  "that",
  ",",
  "in",
  "the",
  "height",
  "of",
  "his",
  "glory",
  ",",
  "he",
  "had",
...

In [4]:
val sortedDistinctTokens = tokenizedText.sorted.distinct

println(s"${sortedDistinctTokens.length} distinct tokens in total")

1130 distinct tokens in total


sortedDistinctTokens: Vector[String] = Vector(
  "!",
  "\"",
  "'",
  "(",
  ")",
  ",",
  "--",
  ".",
  ":",
  ";",
  "?",
  "A",
  "Ah",
  "Among",
  "And",
  "Are",
  "Arrt",
  "As",
  "At",
  "Be",
  "Begin",
  "Burlington",
  "But",
  "By",
  "Carlo",
  "Chicago",
  "Claude",
  "Come",
  "Croft",
  "Destroyed",
  "Devonshire",
  "Don",
  "Dubarry",
  "Emperors",
  "Florence",
  "For",
  "Gallery",
  "Gideon",
...

In [5]:
val vocabulary = sortedDistinctTokens.zipWithIndex.toMap

class SimpleTokenizerV1(
  vocabulary: Map[String, Int]
) {
  val inverseVocabulary = vocabulary.map(_.swap)

  def encode(text: String): Vector[Int] = 
    tokenize(text).map(vocabulary(_))

  def tokenize(text: String): Vector[String] = {
    val splitBy = """[,.:;?_!"()\']|--|\s"""
    val tokenizer = s"(?<=$splitBy)|(?=$splitBy)"
    text.split(tokenizer).filter(!_.isBlank).toVector
  }

  def decode(ids: Vector[Int]): String = 
    ids
      .map(inverseVocabulary(_))
      .mkString(" ")
      .replaceAll("\\s+([,.?!\"()\'])", "$1") 
}

vocabulary: Map[String, Int] = HashMap(
  "inevitable" -> 571,
  "Monte" -> 64,
  "down" -> 362,
  "economy" -> 377,
  "interesting" -> 578,
  "luxury" -> 652,
  "serious" -> 870,
  "forgotten" -> 463,
  "muscles" -> 695,
  "beneath" -> 215,
  "used" -> 1057,
  "eye" -> 415,
  "straining" -> 934,
  "At" -> 18,
  "hooded" -> 554,
  "murmur" -> 694,
  "adulation" -> 133,
  "gloried" -> 495,
  "widow" -> 1102,
  "panel" -> 752,
  "sitters" -> 898,
  "quality" -> 808,
  "On" -> 75,
  "Has" -> 47,
  "secret" -> 861,
  "Money" -> 63,
  "ourselves" -> 737,
  "able" -> 117,
  "bravura" -> 226,
  "behind" -> 212,
  "failure" -> 423,
  "deprecating" -> 327,
  "for" -> 456,
  "canvases" -> 246,
  "wits" -> 1110,
  "s" -> 850,
  "twenty-four" -> 1039,
  "paint" -> 745,
...
defined class SimpleTokenizerV1

In [6]:
val tokenizer = new SimpleTokenizerV1(vocabulary)

val textToEncode = """"It's the last he painted, you know,"
Mrs. Gisburn said with pardonable pride."""
val ids = tokenizer.encode(textToEncode)
val decodedText = tokenizer.decode(ids)

println(decodedText)

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


tokenizer: SimpleTokenizerV1 = ammonite.$sess.cmd5$Helper$SimpleTokenizerV1@40e94690
textToEncode: String = """"It's the last he painted, you know,"
Mrs. Gisburn said with pardonable pride."""
ids: Vector[Int] = Vector(
  1,
  56,
  2,
  850,
  988,
  602,
  533,
  746,
  5,
  1126,
  596,
  5,
  1,
  67,
  7,
  38,
  851,
  1108,
  754,
  793,
  7
)
decodedText: String = "\" It' s the last he painted, you know,\" Mrs. Gisburn said with pardonable pride."

In [7]:
val endOfText = "<|endoftext|>"
val vocabularyWithEndOfText = (sortedDistinctTokens :+ endOfText).zipWithIndex.toMap
val (unknownTokenId, unknownToken) = -1 -> "<|unknown|>"

class SimpleTokenizerV2(
  vocabulary: Map[String, Int]
) {
  val inverseVocabulary = vocabulary.map(_.swap)

  def encode(text: String): Vector[Int] = 
    tokenize(text).map(vocabulary.getOrElse(_, unknownTokenId))

  def tokenize(text: String): Vector[String] = {
    val splitBy = """[,.:;?_!"()\']|--|\s"""
    val tokenizer = s"(?<=$splitBy)|(?=$splitBy)"
    text.split(tokenizer).filter(!_.isBlank).toVector
  }

  def decode(ids: Vector[Int]): String = 
    ids
      .map(inverseVocabulary.getOrElse(_, unknownToken))
      .mkString(" ")
      .replaceAll("\\s+([,.?!\"()\'])", "$1") 
}

endOfText: String = "<|endoftext|>"
vocabularyWithEndOfText: Map[String, Int] = HashMap(
  "inevitable" -> 571,
  "Monte" -> 64,
  "down" -> 362,
  "economy" -> 377,
  "interesting" -> 578,
  "luxury" -> 652,
  "serious" -> 870,
  "forgotten" -> 463,
  "muscles" -> 695,
  "beneath" -> 215,
  "used" -> 1057,
  "eye" -> 415,
  "straining" -> 934,
  "At" -> 18,
  "hooded" -> 554,
  "murmur" -> 694,
  "adulation" -> 133,
  "gloried" -> 495,
  "widow" -> 1102,
  "panel" -> 752,
  "sitters" -> 898,
  "quality" -> 808,
  "On" -> 75,
  "Has" -> 47,
  "secret" -> 861,
  "Money" -> 63,
  "ourselves" -> 737,
  "able" -> 117,
  "bravura" -> 226,
  "behind" -> 212,
  "failure" -> 423,
  "deprecating" -> 327,
  "for" -> 456,
  "canvases" -> 246,
  "wits" -> 1110,
  "s" -> 850,
  "twenty-four" -> 1039,
  "paint" -> 745,
...
unknownTokenId: Int = -1
unknownToken: String = "<|unknown|>"
defined class SimpleTokenizerV2

In [8]:
val tokenizerV2 = new SimpleTokenizerV2(vocabulary)

val concatenatedText = "Hello, do you like tea?" + s" $endOfText " + "In the sunlit terraces of the palace."

import scala.util.chaining._
println(concatenatedText.pipe(tokenizerV2.encode).pipe(tokenizerV2.decode))

<|unknown|>, do you like tea? <|unknown|> In the sunlit terraces of the <|unknown|>.


tokenizerV2: SimpleTokenizerV2 = ammonite.$sess.cmd7$Helper$SimpleTokenizerV2@77d6290b
concatenatedText: String = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace."
import scala.util.chaining._

In [9]:
import $ivy.`dev.scalapy::scalapy-core:0.5.3`

Downloaded https://repo1.maven.org/maven2/dev/scalapy/scalapy-core_2.13/0.5.3/scalapy-core_2.13-0.5.3.pom
Downloaded https://repo1.maven.org/maven2/sh/almond/almond-scalapy_2.13/0.14.0-RC15/almond-scalapy_2.13-0.14.0-RC15.pom
Downloaded https://repo1.maven.org/maven2/dev/scalapy/scalapy-macros_2.13/0.5.3/scalapy-macros_2.13-0.5.3.pom
Downloaded https://repo1.maven.org/maven2/sh/almond/almond-scalapy_2.13/0.14.0-RC15/almond-scalapy_2.13-0.14.0-RC15-sources.jar
Downloaded https://repo1.maven.org/maven2/dev/scalapy/scalapy-macros_2.13/0.5.3/scalapy-macros_2.13-0.5.3-sources.jar
Downloaded https://repo1.maven.org/maven2/dev/scalapy/scalapy-core_2.13/0.5.3/scalapy-core_2.13-0.5.3-sources.jar
Downloaded https://repo1.maven.org/maven2/dev/scalapy/scalapy-macros_2.13/0.5.3/scalapy-macros_2.13-0.5.3.jar
Downloaded https://repo1.maven.org/maven2/sh/almond/almond-scalapy_2.13/0.14.0-RC15/almond-scalapy_2.13-0.14.0-RC15.jar
Downloaded https://repo1.maven.org/maven2/dev/scalapy/scalapy-core_2.13/0.

import $ivy.$

In [10]:
import $file.^.Magic

Compiling /workspace/Magic.sc


import $file.$

In [11]:
Magic.!("pip", "install", "tiktoken==0.7.*")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 795.0/795.0 kB 1.9 MB/s eta 0:00:00


In [12]:
import me.shadaj.scalapy.py

val tiktoken = py.module("tiktoken")

import me.shadaj.scalapy.py
tiktoken: py.Module = <module 'tiktoken' from '/usr/local/lib/python3.12/site-packages/tiktoken/__init__.py'>

In [13]:
import py.SeqConverters

val tiktokenizer = tiktoken.get_encoding("gpt2")
val tiktext = s"Hello, do you like tea? $endOfText In the sunlit terraces of someunknownPlace"
val allowedSpecial = py.Dynamic.global.set(Seq(endOfText).toPythonProxy)
val tiktokens = tiktokenizer.encode(tiktext, allowed_special = allowedSpecial).as[Vector[Int]]
val decodedTiktokens = tiktokenizer.decode(tiktokens.toPythonProxy).as[String]

println(decodedTiktokens)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace


import py.SeqConverters
tiktokenizer: py.Dynamic = <Encoding 'gpt2'>
tiktext: String = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace"
allowedSpecial: py.Dynamic = {'<|endoftext|>'}
tiktokens: Vector[Int] = Vector(
  15496,
  11,
  466,
  345,
  588,
  8887,
  30,
  220,
  50256,
  554,
  262,
  4252,
  18250,
  8812,
  2114,
  286,
  617,
  34680,
  27271
)
decodedTiktokens: String = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace"

In [14]:
// Exercise 2.1
val unknownWords = "Akwirw ier"
println(s"Input: $unknownWords")

val encodedUnknownWords = tiktokenizer.encode(unknownWords).as[Vector[Int]]
val encoding = encodedUnknownWords.map(int => int -> tiktokenizer.decode(Seq(int).toPythonProxy).as[String]).toMap

println("Encoding: ")
encoding.toList.sortBy { case (_, subword) => subword }.foreach { case (int, subword) =>
  println(s"  \"$subword\" -> $int")
}

val decodedUnknownWords = tiktokenizer.decode(encodedUnknownWords.toPythonProxy).as[String]
println(s"Decoded: $decodedUnknownWords")

Input: Akwirw ier
Encoding: 
  " " -> 220
  "Ak" -> 33901
  "ier" -> 959
  "ir" -> 343
  "w" -> 86
Decoded: Akwirw ier


unknownWords: String = "Akwirw ier"
encodedUnknownWords: Vector[Int] = Vector(33901, 86, 343, 86, 220, 959)
encoding: Map[Int, String] = HashMap(
  220 -> " ",
  33901 -> "Ak",
  343 -> "ir",
  86 -> "w",
  959 -> "ier"
)
decodedUnknownWords: String = "Akwirw ier"

In [15]:
val rawTextTokenized = tiktokenizer.encode(rawText).as[Vector[Int]]

println(s"Raw text token count: ${rawTextTokenized.length}")

val rawTextTokensSampled = rawTextTokenized.drop(50)

val contextSize = 4

(1 to contextSize).foreach { size =>
  val context = rawTextTokensSampled.take(size)
  val desired = rawTextTokensSampled(size)
  println(s"${tiktokenizer.decode(context.toPythonProxy)} ---> ${tiktokenizer.decode(Seq(desired).toPythonProxy)}")
}

Raw text token count: 5145
 and --->  established
 and established --->  himself
 and established himself --->  in
 and established himself in --->  a


rawTextTokenized: Vector[Int] = Vector(
  40,
  367,
  2885,
  1464,
  1807,
  3619,
  402,
  271,
  10899,
  2138,
  257,
  7026,
  15632,
  438,
  2016,
  257,
  922,
  5891,
  1576,
  438,
  568,
  340,
  373,
  645,
  1049,
  5975,
  284,
  502,
  284,
  3285,
  326,
  11,
  287,
  262,
  6001,
  286,
  465,
  13476,
...
rawTextTokensSampled: Vector[Int] = Vector(
  290,
  4920,
  2241,
  287,
  257,
  4489,
  64,
  319,
  262,
  34686,
  41976,
  13,
  357,
  10915,
  314,
  2138,
  1807,
  340,
  561,
  423,
  587,
  10598,
  393,
  28537,
  2014,
  198,
  198,
  1,
  464,
  6001,
  286,
  465,
  13476,
  1,
  438,
  5562,
  373,
  644,
...
contextSize: Int = 4

In [16]:
Magic.!("pip", "install", "torch==2.4.*")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.6/89.6 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 22.7 MB/s eta 0:00:00


In [17]:
import me.shadaj.scalapy.interpreter.CPythonInterpreter

val torch = py.module("torch")

// Workaround to define a class that inherits from a Python class
CPythonInterpreter.execManyLines {
  s"""from torch.utils.data import Dataset
     |
     |class GPTDatasetV1(Dataset):
     |  def __init__(self, input_tokens, target_tokens):
     |    self.input_tokens = input_tokens
     |    self.target_tokens = target_tokens
     |  
     |  def __len__(self):
     |    return len(self.input_tokens)
     |
     |  def __getitem__(self, index):
     |    return self.input_tokens[index], self.target_tokens[index]
     |""".stripMargin
}
type Tokenizer = py.Dynamic
type TorchTensor = py.Dynamic
def GPTDatasetV1(
  text: String,
  tokenizer: Tokenizer,
  maxLength: Int,
  step: Int
): py.Dynamic = {
  val tokens = tokenizer.encode(text).as[Vector[Int]]
  val (inputTokens, outputTokens) = (0 until tokens.length by step).foldLeft(
    (
      Vector.empty[TorchTensor], 
      Vector.empty[TorchTensor]
    )
  ) {
    case ((inputTokens, outputTokens), i) =>
      val inputChunk = tokens.slice(i, i + maxLength)
      val outputChunk = tokens.slice(i + 1, i + 1 + maxLength)
      (
       inputTokens :+ torch.tensor(inputChunk.toPythonProxy), 
       outputTokens :+ torch.tensor(outputChunk.toPythonProxy)
      )
  }
  py.Dynamic.global.GPTDatasetV1(inputTokens.toPythonProxy, outputTokens.toPythonProxy)
}

def createDataLoaderV1(
  text: String, 
  batchSize: Int = 4, 
  maxLength: Int = 256,                       
  step: Int = 128, 
  shuffle: Boolean = true, 
  dropLast: Boolean = true,
  numWorkers: Int = 0 
): py.Dynamic = {
  val tokenizer = tiktoken.get_encoding("gpt2")
  val dataset = GPTDatasetV1(text, tokenizer, maxLength, step)
  torch.utils.data.DataLoader(
    dataset,
    batch_size = batchSize,
    shuffle = shuffle,
    drop_last = dropLast,
    num_workers = numWorkers
  )
}

import me.shadaj.scalapy.interpreter.CPythonInterpreter
torch: py.Module = <module 'torch' from '/usr/local/lib/python3.12/site-packages/torch/__init__.py'>
defined type Tokenizer
defined type TorchTensor
defined function GPTDatasetV1
defined function createDataLoaderV1

In [18]:
val dataLoader = createDataLoaderV1(
  text = rawText, 
  batchSize = 1, 
  maxLength = 4, 
  step = 1, 
  shuffle = false
)
val dataLoaderIterator = py.Dynamic.global.iter(dataLoader)
val firstBatch = py.Dynamic.global.next(dataLoaderIterator)
val secondBatch = py.Dynamic.global.next(dataLoaderIterator)
println(firstBatch)
println(secondBatch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


dataLoader: py.Dynamic = <torch.utils.data.dataloader.DataLoader object at 0xfffe93433770>
dataLoaderIterator: py.Dynamic = <torch.utils.data.dataloader._SingleProcessDataLoaderIter object at 0xfffe93589f70>
firstBatch: py.Dynamic = [tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
secondBatch: py.Dynamic = [tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]

In [19]:
// Exercise 2.2
def printFirst2Batches(maxLength: Int, step: Int) = {
  println(s"maxLength = $maxLength, step = $step")
  val dataLoader = createDataLoaderV1(
    text = rawText, 
    batchSize = 1, 
    maxLength = maxLength, 
    step = step, 
    shuffle = false
  )
  val dataLoaderIterator = py.Dynamic.global.iter(dataLoader)
  val firstBatch = py.Dynamic.global.next(dataLoaderIterator)
  val secondBatch = py.Dynamic.global.next(dataLoaderIterator)
  println(firstBatch)
  println(secondBatch)
}

printFirst2Batches(maxLength = 2, step = 2)
printFirst2Batches(maxLength = 8, step = 2)

maxLength = 2, step = 2
[tensor([[ 40, 367]]), tensor([[ 367, 2885]])]
[tensor([[2885, 1464]]), tensor([[1464, 1807]])]
maxLength = 8, step = 2
[tensor([[  40,  367, 2885, 1464, 1807, 3619,  402,  271]]), tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899]])]
[tensor([[ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138]]), tensor([[ 1464,  1807,  3619,   402,   271, 10899,  2138,   257]])]


defined function printFirst2Batches

In [20]:
val batchedDataloader = createDataLoaderV1(
  text = rawText, 
  batchSize = 8, 
  maxLength = 4, 
  step = 4, // same as maxLength to prevent overlaps between inputs and outputs
  shuffle = false
)
val Seq(inputTokens, outputTokens) = py.Dynamic.global.next(py.Dynamic.global.iter(batchedDataloader)).as[Seq[TorchTensor]]
println(s"Inputs:\n$inputTokens")
println(s"Outputs:\n$outputTokens")

Inputs:
tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Outputs:
tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


batchedDataloader: py.Dynamic = <torch.utils.data.dataloader.DataLoader object at 0xfffe986167e0>
inputTokens: TorchTensor = tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
outputTokens: TorchTensor = tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])

In [21]:
val vocabularySize = 50257 // tiktoken vocabulary size
val outputDimension = 256
val tokenEmbeddingLayer = torch.nn.Embedding(vocabularySize, outputDimension)

val maxLength = 4
val dataLoader = createDataLoaderV1(
  text = rawText, 
  batchSize = 8, 
  maxLength = maxLength, 
  step = maxLength,
  shuffle = false
)
val Seq(inputTokensBatchToEmbed, _) = py.Dynamic.global.next(py.Dynamic.global.iter(dataLoader)).as[Seq[TorchTensor]]
println(s"Input batch to embed:\n$inputTokensBatchToEmbed")

val inputTokensEmbedding = tokenEmbeddingLayer(inputTokensBatchToEmbed)
println(s"Input embedding shape: ${inputTokensEmbedding.shape}")

Input batch to embed:
tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Input embedding shape: torch.Size([8, 4, 256])


vocabularySize: Int = 50257
outputDimension: Int = 256
tokenEmbeddingLayer: py.Dynamic = Embedding(50257, 256)
maxLength: Int = 4
dataLoader: py.Dynamic = <torch.utils.data.dataloader.DataLoader object at 0xfffe93f02870>
inputTokensBatchToEmbed: TorchTensor = tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
inputTokensEmbedding: py.Dynamic = tensor([[[-7.5324e-01,  1.4207e+00,  1.1090e-01,  ...,  1.1820e+00,
          -9.7022e-01,  3.3539e-02],
         [-1.9319e+00,  2.4897e-01, -7.4391e-01,  ..., -4.6399e-01,
          -5.6382e-01, -1.3706e+00],
         [-5.9705e-01, -4.9029e-02, -1.9657e+00,  ...,  2.1607e-02,
          -8.0000e-01,  2.5539e+00],
         [ 1.8441e+00,  2.0269e-01, -9.0610e-01,  ..., -5.3811e-01,
           7.3627

In [22]:
val contextLength = maxLength
val positionalEmbeddingLayer = torch.nn.Embedding(contextLength, outputDimension)
val positionalEmbedding = positionalEmbeddingLayer(torch.arange(contextLength))
println(s"Positional embedding shape: ${positionalEmbedding.shape}")

Positional embedding shape: torch.Size([4, 256])


contextLength: Int = 4
positionalEmbeddingLayer: py.Dynamic = Embedding(4, 256)
positionalEmbedding: py.Dynamic = tensor([[-0.3846,  0.3801,  0.3639,  ...,  0.2006, -1.9019, -0.7987],
        [-0.2371, -0.7840, -0.6029,  ...,  0.7719, -0.4472,  1.4853],
        [ 0.3948,  0.2845, -1.8637,  ...,  0.1639,  1.8620,  0.4120],
        [-0.3974, -1.2032,  1.8989,  ..., -0.3128, -0.7291, -1.0689]],
       grad_fn=<EmbeddingBackward0>)

In [23]:
import py.PyQuote

val combinedEmbedding = py"$inputTokensEmbedding + $positionalEmbedding"
println(s"Combined embedding shape: ${combinedEmbedding.shape}")

Combined embedding shape: torch.Size([8, 4, 256])


import py.PyQuote
combinedEmbedding: py.Dynamic = tensor([[[-1.1378e+00,  1.8008e+00,  4.7484e-01,  ...,  1.3826e+00,
          -2.8721e+00, -7.6515e-01],
         [-2.1690e+00, -5.3505e-01, -1.3468e+00,  ...,  3.0795e-01,
          -1.0110e+00,  1.1464e-01],
         [-2.0230e-01,  2.3546e-01, -3.8294e+00,  ...,  1.8550e-01,
           1.0620e+00,  2.9659e+00],
         [ 1.4467e+00, -1.0005e+00,  9.9284e-01,  ..., -8.5095e-01,
           7.1353e-03,  3.0498e-01]],

        [[ 1.0937e-01, -2.0513e-03, -3.4637e-01,  ...,  7.5539e-02,
          -1.9414e+00, -7.0576e-01],
         [ 8.1356e-01, -2.0920e+00, -1.1638e+00,  ...,  3.0826e+00,
          -3.0113e-01,  2.4515e+00],
         [ 1.0292e+00, -1.7144e+00, -3.3554e+00,  ...,  1.6538e-01,
           1.3030e+00,  1.2663e+00],
         [-7.7631e-01, -1.4665e+00,  1.8170e+00,  ...,  6.9816e-01,
          -1.6878e-01, -1.5186e+00]],

        [[-1.0886e+00,  2.1670e+00,  6.5620e-01,  ..., -1.3898e+00,
          -2.3450e+00,  9.2697e-01],
 